In [ ]:
# import packages
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.utils.data as Data

import pandas as pd
import numpy as np
from torchtext.data.utils import get_tokenizer
import torch
import torchtext
import random
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from torch import nn 
import torch.nn.functional as F

from torch import utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.metrics import f1_score

from torch.utils.data import Dataset

from torchtext.vocab import vocab
from collections import Counter

from torchtext.transforms import Truncate
from torch.nn.utils.rnn import pad_sequence

from torch.nn.utils.rnn import pad_sequence


np.random.seed(0)

In [ ]:
# read train data
train=pd.read_csv('train.csv').copy()

#combine all the text features 
train['text']=train['summary']+' '+train['reviewText']  
text=train['text']

#The result of classification will start from zero rather than 1, therefore the ratings are re-coded here for convinence in the following steps.
train['rating'].unique() 
train['response']=train['rating']-1
train.drop(['reviewText','summary','rating'],axis=1,inplace=True)

# read test data
test=pd.read_csv('test.csv').copy()
test['text']=test['summary']+' '+test['reviewText'] #combine all the text features  
test.drop(['reviewText','summary','Id'],axis=1,inplace=True)

# split avaliable data to train data and validation data
df_b=train.copy()
ind_train_b, ind_valid_b  = train_test_split(np.array(df_b.index), train_size=0.9, random_state=1)
train_b = df_b.loc[ind_train_b,:].copy()
valid_b = df_b.loc[ind_valid_b,:].copy()
train_b

# define class to change raw data to torch dataset
class MyDataset(Dataset):
    def __init__(self, myData):
        super().__init__()
        self.data = myData     

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.data.shape[1]==2:
            return (self.data.iloc[idx,0], self.data.iloc[idx,1]) # (text, response)(for avaliable data)
        else:
            return (self.data.iloc[idx,0])#(only text)(for test data)

# turn original dataset to torch dataset 
train_torch = MyDataset(train_b)
val_torch = MyDataset(valid_b)
test_torch = MyDataset(test)


In [ ]:
# loading nltk packages about natural language processing
import nltk
from nltk.stem import SnowballStemmer
nltk.download('wordnet')
nltk.download('stopwords')
stpwords = stopwords.words('english')
stemmer = SnowballStemmer('english')
WNL=WordNetLemmatizer()
basic_tokenizer = get_tokenizer('basic_english')

# prepare regular expression for data cleaning 
punctuations = [',', '.', ':', ';', '<','>', '(', ')', '[', ']', '&','/','{','}','-', '+','-', '*', '@', '#', '$', '%',"'"]
clean_patterns = [(r'won\'t','will not'),(r'can\'t','cannot'),(r'i\'m','i am'),
                       (r'ain\'t','is not'),(r'(\w+)\'ll','\g<1> will'),(r'(\w+)n\'t','\g<1> not'),
                       (r'(\w+)\'ve','\g<1> have'),(r'(\w+)\'s','\g<1> is'),(r'(\w+)\'re','\g<1> are'),
                       (r'(\w+)\'d','\g<1> would'),(r'&','and'),
                       (r'dont','do not'),(r'wont','will not'),('[0-9]+',''),(r'(\!)\1+',' multihighlight'),(r'(\?)\1+', ' multiwhatt'),]

# define function to do initial data cleaning and tokenization
def my_tokenizer(text):
    text=re.sub('[0-9]+','',text) #Get rid of numbers
    text=re.sub(r"(\!)\1+",' multiemphasize',text)
    text=re.sub(r"(\?)\1+",' doubleconfused',text)
    text=re.sub(r"(\.)\1+",' multistop',text)

    patterns=[(re.compile(regex),repl)for (regex,repl) in clean_patterns]
    for (pattern,repl)in patterns:
        (text,count)=re.subn(pattern,repl,text)
    
    split_text=basic_tokenizer(text) #It converts all letters to lowercase and separates punctuation from words
    filter_punc=[word for word in split_text if word not in punctuations]#Get rid of the punctuations except "!" and "?" because they have some emotional tendency
    # filter_stop=[word for word in filter_punc if word not in stpwords] #this code is abandoned beacuse our model performs better with stop words
    text=[WNL.lemmatize(word) for word in filter_punc]#lemmatize words
    text2=[stemmer.stem(word) for word in text]#stem words
    return text2

# test defined function
print(my_tokenizer("i'm your brother!!!!! and i want to love you 4000 times?????? ...... can't do that"))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'am', 'your', 'brother', 'multiemphas', 'and', 'i', 'want', 'to', 'love', 'you', 'time', 'doubleconfus', 'multistop', 'cannot', 'do', 'that']


In [ ]:
# activate GPU to assist model training
print(torch.cuda.is_available())
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# prepare list for hyperparameter tuning
datadata = []
innerdata = []

# training model
# step1: select appropriate batchsize, learning rate, hidden dim, embed dim (since we can not change all hyperparameter each time)
# step2: prepare lists for min_fre and pad_length to check whether they are good for the model 
for voc_min_fre in [16]: # other possible value such as 8,20,24 were deleted and most suitable value is kept as 16
    for pad_length in [300]: # other possible value such as 128,160,180,256 were deleted and most suitable value is 300

        # update vocabulary
        counter = Counter() 
        for doc in df_b['text']:
            counter.update(my_tokenizer(doc))
        vocabulary = vocab(counter, min_freq = voc_min_fre) 
        vocab_size=len(vocabulary)

        # default index is 0
        vocabulary.set_default_index(0)
  
        # define function that turn tokenized comment word to number
        # predefined data cleaning and tokenization function is used
        def doc_tokenizer(doc):
            idxoftoken=[vocabulary[token] for token in my_tokenizer(doc)]
            return (idxoftoken)
        
        # define the maximum length for each tokenized comment
        truncate = Truncate(max_seq_len=pad_length)  

        # define collate_fn, which intergrate previous methods that convert raw text data to trunctuated numerical data
        def collate_batch_advanced(batch):
            text_list,target_list = [], []
            for idx in range(len(batch)):
                _label = batch[idx][1]
                _text = batch[idx][0]
                target_list.append( _label )
                tokens = doc_tokenizer(_text)
                truncate_token=truncate(tokens)
                truncated_tokens=torch.tensor(truncate_token, dtype=torch.long)
                text_list.append(truncated_tokens)
            target_list = torch.tensor(target_list, dtype=torch.int64)
            return text_list,target_list
        
        # define collate_fn for test dataloader without label
        def test_tokenizer(batch):
            text_list = []
            for idx in range(len(batch)):
                _text = batch[idx][0]
                tokens = doc_tokenizer(_text)
                truncate_token=truncate(tokens)
                truncated_tokens=torch.tensor(truncate_token, dtype=torch.long)
                text_list.append(truncated_tokens)
            return text_list

        # padding trunctuated numerical data to matrix
        input_x =[[1,2,3],[4,5,6,7,8],[8,9]]
        norm_data_pad = pad_sequence([torch.from_numpy(np.array(x)) for x in input_x], batch_first=True).float()
        norm_data_pad

        # define evaluation function which return predicted values by trained model and true values
        def evaluate_adv(dataloader):
            y_pred = torch.tensor([]).to(device) 
            y_true = torch.tensor([]).to(device) 
            model.eval()
            with torch.no_grad():
                for text, label in dataloader:
                    text = pad_sequence(text, batch_first=True, padding_value=0).to(device)
                    y_pred_batch = model(text).to(device)
                    y_pred = torch.cat((y_pred, y_pred_batch.squeeze()))
                    y_true = torch.cat((y_true, label.squeeze().to(device)))
            return y_pred, y_true

        # set random seed
        # this random seed may change to check whether one model can also get high score under different seed
        torch.manual_seed(0)
        # torch.cuda.manual_seed(0)

        # set batch size for dataloader
        batchSize = 64
        batch_size = 64
        train_loader = utils.data.DataLoader(train_torch, batch_size=batchSize, shuffle=True, collate_fn=collate_batch_advanced)
        val_loader = utils.data.DataLoader(val_torch, batch_size=batchSize, shuffle=True, collate_fn=collate_batch_advanced)

        # define the final model: Attention based bidirectional LSTM 
        class BiLSTM_Attention(nn.Module):
            def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
                super().__init__()
                self.hidden_dim = hidden_dim       
                self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
                self.lstm = nn.LSTM(embed_dim, hidden_dim, bidirectional=True,batch_first=True)
                self.linear = nn.Linear(hidden_dim*2, output_dim)
                self.Dropout = nn.Dropout(p=0.4)
                # dropout is another hyperparameter, normally, set it from 0.2 to 0.6 is acceptable for avoid overfitting

            def attention_net(self, lstm_output, final_state): #attention layer is designed to improve the performance
                batch_size = len(lstm_output)
                hidden = final_state.view(batch_size, -1, 1)  
                attn_weights = torch.bmm(lstm_output, hidden).squeeze(2) 
                soft_attn_weights = F.softmax(attn_weights, 1)
                context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
                return context, soft_attn_weights 

            def forward(self, X):
                input = self.embedding(X) 
                output, (final_hidden_state, final_cell_state) = self.lstm(input)
                attn_output, attention = self.attention_net(output, final_hidden_state) # output from lstm need to go attention layer
                attn_output = self.Dropout(attn_output)
                return self.linear(attn_output.squeeze(0)) # finally, output layer is derived from attention layer output

        # hyperparameter for model design 
        # decided by trial and error to find good model design
        embed_dim = 256
        hidden_dim = 128
        output_dim = 5
        num_LstmLayer = 1
        
        # Prepare for training the model
        model = BiLSTM_Attention(vocab_size, embed_dim, hidden_dim, output_dim).to(device)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.002)
        
        # Prepare list to save validation score and error under each epoch
        score_test_list = []
        error_test_list = []
        
        # train model in predetermined epochs
        epochs = 8
        for epoch in range(epochs): 
            for text,y_train in train_loader:
                model.train()
                optimizer.zero_grad()
                text = pad_sequence(text, batch_first=True, padding_value=0) # pad the sequences with 0
                text=text.to(device)
                y_train=y_train.to(device)
                y_pred_train = model(text).to(device)
                loss = loss_fn(y_pred_train.view(-1, y_pred_train.shape[-1]), y_train.view(-1))
                loss.backward()
                optimizer.step()

            # evaluate on validation set
            y_pred_val, y_val = evaluate_adv(val_loader)
            loss_val = loss_fn(y_pred_val.squeeze(), y_val.long())
            pred_label = torch.argmax( y_pred_val, -1).long() # find out the class prediction [0-4]
            acc = (pred_label == y_val).float().sum()/y_val.shape[0]
            f1 = f1_score(pred_label.cpu(), y_val.cpu().numpy(), average = 'macro')
            print('Epoch {}: {:.4f} (train), {:.4f} (val), {:.4f} (val acc),{:.4f}(f1)'.format(epoch, loss, loss_val, acc,f1))

            # save detailed performance data to lists
            score_test_list.append(f1)
            error_test_list.append(loss_val)
            innerdata.append(f1)

            # save related hyperparameter information
            if epoch == 9:
                innerdata.append('almost')
                innerdata.append(voc_min_fre)
                innerdata.append(pad_length)
                innerdata.append('split-----')
                datadata.append(innerdata)
                innerdata = []
                print('one lap over')

# permanently save the information about model and hyperparameter
final_list = np.asarray(datadata)
data_df = pd.DataFrame(final_list)
data_df.to_csv('123attendropout.csv')

True
Epoch 0: 1.4746 (train), 1.4283 (val), 0.3789 (val acc),0.3131(f1)
Epoch 1: 1.2425 (train), 1.1957 (val), 0.4667 (val acc),0.3786(f1)
Epoch 2: 1.1074 (train), 1.2224 (val), 0.4711 (val acc),0.4214(f1)
Epoch 3: 0.9416 (train), 1.1502 (val), 0.5044 (val acc),0.4707(f1)
Epoch 4: 0.9838 (train), 1.1627 (val), 0.5200 (val acc),0.5001(f1)
Epoch 5: 0.8415 (train), 1.1602 (val), 0.5244 (val acc),0.5117(f1)
Epoch 6: 0.9907 (train), 1.2217 (val), 0.5178 (val acc),0.5074(f1)
Epoch 7: 0.4425 (train), 1.2977 (val), 0.5322 (val acc),0.5214(f1)


In [5]:
# use all avalible data to train the model
# make use of majority of votes
# ep n is the epoch has least validation error in previous part

for random_seed in [0,258,777,888,999,1001,1222,1345,1500,1600,1700,1800,1999,2000,2016,2020,2022,2077]:
  for epoch_num in [6]:

    all_torch=MyDataset(train)
    all_loader = utils.data.DataLoader(all_torch, batch_size=batchSize, shuffle=True, collate_fn=collate_batch_advanced)

    torch.manual_seed(random_seed)

    # same function
    def test_tokenizer(batch):
        text_list = []
        for idx in range(len(batch)):
            _text = batch[idx]
            tokens = doc_tokenizer(_text)
            truncate_token=truncate(tokens)
            truncated_tokens=torch.tensor(truncate_token, dtype=torch.long)
            text_list.append(truncated_tokens)
        return text_list

    all_torch=MyDataset(train)
    all_loader = utils.data.DataLoader(all_torch, batch_size=batchSize, shuffle=True, collate_fn=collate_batch_advanced)

    # same model
    class BiLSTM_Attention(nn.Module):
        def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
            super().__init__()
            self.hidden_dim = hidden_dim       
            self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
            self.lstm = nn.LSTM(embed_dim, hidden_dim, bidirectional=True,batch_first=True)
            self.linear = nn.Linear(hidden_dim*2, output_dim)
            self.Dropout = nn.Dropout(p=0.4)
        # lstm_output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
        
        def attention_net(self, lstm_output, final_state):
            batch_size = len(lstm_output)
            hidden = final_state.view(batch_size, -1, 1)   # hidden : [batch_size, n_hidden * num_directions(=2), n_layer(=1)]
            attn_weights = torch.bmm(lstm_output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
            soft_attn_weights = F.softmax(attn_weights, 1)
            # context : [batch_size, n_hidden * num_directions(=2)]
            context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)

            return context, soft_attn_weights 
        
        
        def forward(self, X):
            input = self.embedding(X) # input : [seq_len, batch_size, embedding_dim]
            # final_hidden_state, final_cell_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
            output, (final_hidden_state, final_cell_state) = self.lstm(input)
            #output = output.transpose(0, 1) # output : [batch_size, seq_len, n_hidden]
            attn_output, attention = self.attention_net(output, final_hidden_state)
            attn_output = self.Dropout(attn_output)
            return self.linear(attn_output.squeeze(0)) # model : [batch_size, num_classes], attention : [batch_size, n_step]
        
    # same hyperparameter
    embed_dim = 256
    hidden_dim = 128
    output_dim = 5
    num_LstmLayer = 1
    vocab_size=len(vocabulary)
    model = BiLSTM_Attention(vocab_size, embed_dim, hidden_dim, output_dim).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.002)

    # same training method
    epochs = epoch_num
    for epoch in range(epochs): 
        for text,y_train in all_loader:
            model.train()
            optimizer.zero_grad()
            
            text = pad_sequence(text, batch_first=True, padding_value=0)
            text=text.to(device)
            y_train=y_train.to(device)

            y_pred_train = model(text).to(device)
            loss = loss_fn(y_pred_train.view(-1, y_pred_train.shape[-1]), y_train.view(-1))

            loss.backward()
            optimizer.step()
        
        print('Epoch {}: {:.4f} (train)'.format(epoch, loss))

    # make use of adjusted function to process test data
    test_loader1 = utils.data.DataLoader(test_torch, batch_size=3000, shuffle=False, collate_fn=test_tokenizer)
    
    # save models
    modeltitle = 'net_ep6_seed' + str(random_seed)+ '.pth'
    torch.save(model,modeltitle)
    
    # make predictions
    model.eval()
    with torch.no_grad():
        y_test_pred_f = torch.tensor([]).to(device) 
        for text in test_loader1:
            text = pad_sequence(text, batch_first=True, padding_value=0).to(device)          
            y_pred_batch = model(text).to(device)
            y_test_pred_f = torch.cat((y_test_pred_f, y_pred_batch.squeeze()))

    pred_label_test = torch.argmax(y_test_pred_f, -1)
    pred_label_test.cpu() + 1
    # add back 1 deducted before

    # save each predictions 
    # note that pandas will automatically treat id as new column and create new index
    # since we are trying to make many different predictions under same model same ep
    # final csv will use mode function and will be created manually
    possibility = pd.read_csv('sample_submission.csv')
    possibility['Prediction'] = pred_label_test.cpu() + 1
    title = 'sub777' + 'seed' + str(random_seed) + 'epoch' + str(epoch_num) + '.csv'
    possibility.to_csv(title)


Epoch 0: 1.2881 (train)
Epoch 1: 1.0839 (train)
Epoch 2: 1.1397 (train)
Epoch 3: 0.8460 (train)
Epoch 4: 0.8939 (train)
Epoch 5: 0.8371 (train)
Epoch 0: 1.1299 (train)
Epoch 1: 0.9531 (train)
Epoch 2: 1.1745 (train)
Epoch 3: 0.9868 (train)
Epoch 4: 0.7231 (train)
Epoch 5: 0.9398 (train)
Epoch 0: 1.3338 (train)
Epoch 1: 1.0964 (train)
Epoch 2: 1.1981 (train)
Epoch 3: 0.9655 (train)
Epoch 4: 0.8433 (train)
Epoch 5: 0.7486 (train)
Epoch 0: 1.5791 (train)
Epoch 1: 1.0775 (train)
Epoch 2: 1.1242 (train)
Epoch 3: 0.9374 (train)
Epoch 4: 0.8807 (train)
Epoch 5: 0.8383 (train)
Epoch 0: 1.3725 (train)
Epoch 1: 1.0002 (train)
Epoch 2: 0.9126 (train)
Epoch 3: 0.7286 (train)
Epoch 4: 0.9408 (train)
Epoch 5: 0.9117 (train)
Epoch 0: 1.4060 (train)
Epoch 1: 1.2212 (train)
Epoch 2: 1.1433 (train)
Epoch 3: 1.0049 (train)
Epoch 4: 1.0707 (train)
Epoch 5: 0.8396 (train)
Epoch 0: 1.5664 (train)
Epoch 1: 1.1394 (train)
Epoch 2: 1.1509 (train)
Epoch 3: 0.8832 (train)
Epoch 4: 0.9051 (train)
Epoch 5: 0.7324 

In [56]:
class 

TypeError: 'hha' object is not callable

In [37]:
def show(name,number):
        return ("my name is {}".format(name+number))
show('x','123')

'my name is x123'